In [1]:
import databricks.koalas as ks
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import string
from pyspark import SparkConf
from pyspark import SparkContext

import re
from langdetect import detect
from pyspark.sql import SparkSession


In [2]:
confSpark = SparkConf().set("spark.driver.bindAddress", "localhost")
spark = SparkContext("local[*]", "appname", conf = confSpark)
#spark = SparkSession.builder.master("local[*]").appName("URBANICITY").getOrCreate()

In [3]:
%run ./geospatial_functions.ipynb
%run ./01_text_similarity_functions.ipynb
%run ./01_text_prep_functions.ipynb

[nltk_data] Downloading package punkt to C:\Users\Salif
[nltk_data]     SAWADOGO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Salif
[nltk_data]     SAWADOGO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
name_column_regex_replace = {r"\'": "", r"\d{5}": "", r"\s+": " "}
address_column_blacklist = []
address_column_regex_replace = {r"\'": "", r"\s+": " ", "avenu ": "av ", "boulevard ": "bd "}


In [7]:
cmd = spark.read.option("header", "True").\
    option("inferSchema", "true").\
    csv("D:/data_quality/data/customer_invoice_tizi_ouzou.csv") [["Client", "LONGITUDE", "LATITUDE", "Nom", "Adresse"]].\
    withColumnRenamed("Client","CUSTOMER_COD")

In [104]:
cmd.show()

+------------+----------------+----------------+--------------------+--------------------+
|CUSTOMER_COD|       LONGITUDE|        LATITUDE|                 Nom|             Adresse|
+------------+----------------+----------------+--------------------+--------------------+
|       72011|4.10326666666666|36.6073166666666|       MANOUCHE AMAR|    RUE TALA ATHMANE|
|       72009|        4.080755|36.6208683333333|       SADAOUI HAMID|       RUE AMIROUCHE|
|       72000|         4.08768|       36.617363|     SARL TOUTE ELLA|     RUE IGHIL HAMOU|
|       71999|         4.04352|        36.69592|    BOUZERD ABDENOUR|  BENI DOUALA CENTRE|
|       71998|        4.178637|       36.768272|           DLIM SAID|RESIDENCE HADJILI...|
|       71997|4.05602833333333|36.7153966666666|      ALIOUT  RAZIKA|     RUE ANAR AMELAL|
|       71996|      4.03790698|      36.7076863|       MOHDEB CHERIF|      VILLAGE LOUDHA|
|       71991|4.07188333333333|        36.64442|          ETS MEDDAD|   LAGAR SIDI MAAMAR|

In [9]:
cmd_golden_uri="C:/Users/Salif SAWADOGO/OneDrive - EQUATORIAL COCA-COLA BOTTLING COMPANY S.L/dynamic segmentation/matching/output/horeca_tz_customer_subset.csv"
cmd_golden_ids = spark.read.option("header", "True").\
               csv(cmd_golden_uri)\
    [["CUSTOMER_COD"]].withColumn("CUSTOMER_COD", F.col("CUSTOMER_COD").cast("integer"))
cmd_golden_ids.show()

+------------+
|CUSTOMER_COD|
+------------+
|       56341|
|       38014|
|       38861|
|       54921|
|       37832|
|       37757|
|       41128|
|       40777|
|       52474|
|       37968|
|       38055|
|       38144|
|       64884|
|       38015|
|       39745|
|       45301|
|       55586|
|       39813|
|       62873|
|       38867|
+------------+
only showing top 20 rows



In [10]:
cmd_golden_ids.count()

198

In [11]:
name_column_blacklist = ["cafe", "cf", "restaurant", "estaurant", "rest", "ag", "ste", "café", "snack", "hotel", "sarl", "rotisserie", "marrakech"]
name_column_regex_replace = {r"\'": "", r"\d{5}": "", r"\s+": " "}
address_column_blacklist = []
address_column_regex_replace = {r"\'": "", r"\s+": " ", "avenu ": "av ", "boulevard ": "bd "}

In [13]:
cmd_golden = cmd.join(cmd_golden_ids, ["CUSTOMER_COD"]) \
                .withColumn("NOM_CLEAN", make_text_prep_func(word_blacklist=name_column_blacklist, regex_replace=name_column_regex_replace)(F.col("Nom"))) \
                .withColumn("ADRESSE_CLEAN", make_text_prep_func(word_blacklist=address_column_blacklist, regex_replace=address_column_regex_replace)(F.col("Adresse")))

cmd_golden.show()

+------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+
|CUSTOMER_COD|       LONGITUDE|        LATITUDE|                 Nom|             Adresse|           NOM_CLEAN|       ADRESSE_CLEAN|
+------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+
|       67250|3.84282333333333|        36.53764|        FOUNES HAKIM|       OUACIF CENTRE|          foun hakim|          ouac centr|
|       65126|4.05923833333333|36.7026533333333|         COFFEE TIME|BVD LES BOUZARD K...|          coffe time|bvd les bouzard k...|
|       65115|         3.95982|36.7239933333333|         RABHI SAMIR|COMMUNE AIN ZAOUI...|         rabhi samir|commun ain zaoui ...|
|       65113|4.36676666666666|36.6663883333333|        SADJI YOUCEF|      MAKOUDA CENTRE|        sadji youcef|       makouda centr|
|       65061|         3.95595|36.7342433333333|        AMAR CHERGUI|

In [14]:
tripadvisor_data_uri ="C:/Users/Salif SAWADOGO/OneDrive - EQUATORIAL COCA-COLA BOTTLING COMPANY S.L/dynamic segmentation/data_acquisition/TripAdvisor/code/output/ta_combined_l3_Algeria_prepped.parquet"

ta = spark.read.option("header", "True").option("inferSchema", "True").parquet(tripadvisor_data_uri)[["id", "name", "address", "location_lat", "location_lon"]] \
          .withColumn("name_CLEAN", make_text_prep_func(word_blacklist=name_column_blacklist, regex_replace=name_column_regex_replace)(F.col("name"))) \
          .withColumn("address_CLEAN", make_text_prep_func(word_blacklist=address_column_blacklist, regex_replace=address_column_regex_replace)(ta_remove_address_tail(F.col("address")))) \
          .withColumn("location_lat", F.col("location_lat").cast("float")) \
          .withColumn("location_lon", F.col("location_lon").cast("float"))
ta[["name","address","location_lat","location_lon","name_CLEAN", "address_CLEAN"]].show()

+--------------------+--------------------+------------+------------+--------------------+--------------------+
|                name|             address|location_lat|location_lon|          name_CLEAN|       address_CLEAN|
+--------------------+--------------------+------------+------------+--------------------+--------------------+
|         Paris Beach|                null|    35.92549|    0.090527|          pari beach|                    |
|    Pecherie Almirez|                null|    35.93473|     0.08014|     pecheri almirez|                    |
|Pizza 447 Mostaganem|                null|    35.92259|      0.1093|pizza 447 mostaganem|                    |
|             algerie|                null|    35.92785|     0.10009|              algeri|                    |
| Bekhchis Mostaganem|                null|    35.91465|    0.056364|  bekhchi mostaganem|                    |
|             Happy M|                null|   35.915577|    0.057128|               happi|              

In [15]:
ta.count()

1237

In [93]:
def match_join(l_sdf,
               l_id,
               l_lon,
               l_lat,
               l_name,
               l_addr,
               r_sdf,
               r_id,
               r_lon,
               r_lat,
               r_name,
               r_addr,
               distance_threshold_m,
              ):
  l_slice = l_sdf[[l_id, l_lon, l_lat, l_name, l_addr]]
  r_slice = r_sdf[[r_id, r_lon, r_lat, r_name, r_addr]]
  
  l_og_cols = l_slice.columns
  r_og_cols = r_slice.columns
  for c in l_og_cols:
    l_slice = l_slice.withColumnRenamed(c, "L_"+c)
  for c in r_og_cols:
    r_slice = r_slice.withColumnRenamed(c, "R_"+c)
  inner_joined = l_slice.join(r_slice, haversine_distance_sdf(F.col("L_"+l_lon), F.col("L_"+l_lat), F.col("R_"+r_lon), F.col("R_"+r_lat)) <= distance_threshold_m, how="inner")
       
 
# withColumn("rank", F.dense_rank().over(Window.partitionBy('L_' + l_id).orderBy(F.col("similarity").desc()))).filter("rank <= 15")
#   l_joined = l_slice.join(inner_joined, l_slice.columns)
  return(inner_joined)

In [94]:
matched = match_join(cmd_golden, "CUSTOMER_COD", "LONGITUDE", "LATITUDE", "NOM_CLEAN", "ADRESSE_CLEAN", ta, "id", "location_lon", "location_lat", "name_CLEAN", "address_CLEAN", 2000)

In [100]:
l_id="CUSTOMER_COD"
l_lon="LONGITUDE"
l_lat="LATITUDE"
l_name="NOM_CLEAN" 
l_addr="ADRESSE_CLEAN"
r_id="id"
r_lon="location_lon"
r_lat="location_lat"
r_name="name_CLEAN"
r_addr="address_CLEAN"
distance_threshold_m=2000

In [101]:
matched=matched.withColumn("name_similarity", compound_similarity_sdf(F.col("L_"+l_name),F.col("R_"+r_name)))

In [102]:
matched=matched.withColumn("dist_m", haversine_distance_sdf(F.col("L_"+l_lon), F.col("L_"+l_lat), F.col("R_"+r_lon), F.col("R_"+r_lat))).\
          withColumn("name_similarity", compound_similarity_sdf(F.col("L_"+l_name),F.col("R_"+r_name))).\
          withColumn("address_similarity", compound_similarity_sdf(F.col("L_"+l_addr),F.col("R_"+r_addr))).\
          withColumn("dist_similarity", (distance_threshold_m - F.col("dist_m"))/distance_threshold_m).\
          withColumn("similarity", F.col("name_similarity")*0.75 + F.col("address_similarity")*0.15 + F.col("dist_similarity")*0.15).withColumn("rank", F.dense_rank().over(Window.partitionBy('L_' + l_id).orderBy(F.col("similarity").desc()))).filter("rank <= 15")

In [103]:
golden_standart_uri="C:/Users/Salif SAWADOGO/OneDrive - EQUATORIAL COCA-COLA BOTTLING COMPANY S.L/dynamic segmentation/matching/output/golden_standardd.csv"
matched.repartition(1).write.mode("overwrite").option("header", "True").csv(golden_standart_uri)

Py4JJavaError: An error occurred while calling o1192.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at o